In [1]:
## define
bastion_ip='192.168.50.20'
bastion_user='admin'
bastion_pass='icair.750'
session="1"
dst_port="1"
src_port="2"

In [2]:


import paramiko
import time
import re

def endwith_input(prompt_str,input_str,sleep=0,verbose=False):
    buff=''
    while not buff.endswith(prompt_str):
        resp = chan.recv(9999).decode("utf8")
        buff += str(resp)
    if verbose : print(buff)  
    time.sleep(sleep)
    chan.send(input_str+"\n")
    return buff


def endwith_exec_search(prompt_str, cmd, search_str, sleep=0, verbose=False):
    buff = ''
    while not buff.endswith(prompt_str):
        resp = chan.recv(9999).decode("utf8")
        buff += str(resp)
    if verbose : print("initb="+buff)  
    time.sleep(sleep)
    chan.send(cmd+ ";sleep 1; echo @@end@@"+"\n")
    while buff.find('@@end@@') < 0:
        resp = chan.recv(9999).decode("utf8")
        buff += str(resp)
    time.sleep(sleep)
    if verbose : print("buff="+buff)  
    if verbose : print("seach_str="+search_str)
    ret=re.search(search_str, buff)    
    if verbose : print("ret="+str(ret))
    if ret == None:
        return False
    else:
        return True
    #ret = True if ret != None else False


def exec_once(cmd):
    stdin, stdout, stderr = ssh.exec_command(cmd)
    for line in stdout:
        print('... ' + line.strip('\n'))

## initial 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy( paramiko.AutoAddPolicy() )
ssh.connect(bastion_ip, username=bastion_user, password=bastion_pass)
chan = ssh.invoke_shell()

In [3]:
## enable
endwith_input('#','configure terminal', verbose=True)
endwith_input('#','interface hu 1/'+dst_port, verbose=True)
endwith_input('#','no switchport', verbose=True)
endwith_input('#','exit', verbose=True)

endwith_input(')#','monitor session '+session,verbose=True)

endwith_input(')#','source hu 1/'+ src_port +' destination hu 1/'+ dst_port +' direction both',verbose=True)
endwith_input(')#','exit',verbose=True)
endwith_input(')#','exit',verbose=True)
ssh.close()

dtn17dell1#
configure terminal
dtn17dell1(conf)#
interface hu 1/1
dtn17dell1(conf-if-hu-1/1)#
no switchport
dtn17dell1(conf-if-hu-1/1)#
exit
dtn17dell1(conf)#
monitor session 1
dtn17dell1(conf-mon-sess-1)#
source hu 1/2 destination hu 1/1 direction both
dtn17dell1(conf-mon-sess-1)#
exit
dtn17dell1(conf)#


In [4]:
## check result
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy( paramiko.AutoAddPolicy() )
ssh.connect(bastion_ip, username=bastion_user, password=bastion_pass)
stdin, stdout, stderr = ssh.exec_command("show monitor session 1")

for line in stdout:
    print(line.strip('\n'))
ssh.close()

dtn17dell1#show monitor session 1
  SessID  Source         Destination         Dir  Mode  Source IP      Dest IP         DSCP  TTL   Drop  Rate
  ------  ------         -----------         ---  ----  ---------      --------        ----  ---   ----  ----
     1   Hu 1/2            Hu 1/1            both Port     N/A            N/A          N/A   N/A   No     N/A
dtn17dell1#


In [5]:
## tear down
ssh.connect(bastion_ip, username=bastion_user, password=bastion_pass)
chan = ssh.invoke_shell()
endwith_input('#','configure terminal', verbose=True)
endwith_input(')#','no monitor session '+session,verbose=True)
endwith_input(')#','end',verbose=True)

ssh.close()

dtn17dell1#
configure terminal
dtn17dell1(conf)#
no monitor session 1
dtn17dell1(conf)#


In [6]:
## check result
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy( paramiko.AutoAddPolicy() )
ssh.connect(bastion_ip, username=bastion_user, password=bastion_pass)
stdin, stdout, stderr = ssh.exec_command("show monitor session 1")

for line in stdout:
    print(line.strip('\n'))


dtn17dell1#show monitor session 1
% Error: No such session exists.
dtn17dell1#


In [7]:
# close
ssh.close()